In [1]:
spark

Spark local-1536898627823: Some(http://gw02.itversity.com:4045)

In [2]:
spark.sparkContext.getConf.getAll.foreach(x=>if(x._2.contains("/proxy/")){println(x._2)})

In [3]:
import breeze.plot._

In [4]:
import java.io.File
import java.time.LocalDate
import java.time.format.DateTimeFormatter

import scala.collection.mutable.ArrayBuffer
import org.apache.spark.mllib.stat.KernelDensity
import org.apache.spark.sql.{Dataset, SparkSession}
import org.apache.spark.sql.functions
import org.apache.spark.util.StatCounter

import org.apache.commons.math3.distribution.ChiSquaredDistribution
import org.apache.commons.math3.distribution.MultivariateNormalDistribution
import org.apache.commons.math3.random.MersenneTwister
import org.apache.commons.math3.stat.correlation.Covariance
import org.apache.commons.math3.stat.regression.OLSMultipleLinearRegression

In [5]:
val start = LocalDate.of(2009, 10, 23)
val end = LocalDate.of(2014, 10, 23)

start = 2009-10-23
end = 2014-10-23


2014-10-23

In [6]:
val stocksDir = new File("/home/kranthidr/data/risk_analysis/factors")
val files = stocksDir.listFiles()

stocksDir = /home/kranthidr/data/risk_analysis/factors
files = Array(/home/kranthidr/data/risk_analysis/factors/NYSEARCA%3AGLD.csv, /home/kranthidr/data/risk_analysis/factors/NYSEARCA%3ACRED.csv, /home/kranthidr/data/risk_analysis/factors/NASDAQ%3ATLT.csv)


[/home/kranthidr/data/risk_analysis/factors/NYSEARCA%3AGLD.csv, /home/kranthidr/data/risk_analysis/factors/NYSEARCA%3ACRED.csv, /home/kranthidr/data/risk_analysis/factors/NASDAQ%3ATLT.csv]

In [7]:
 def readGoogleHistory(file: File): Array[(LocalDate, Double)] = {
    val formatter = DateTimeFormatter.ofPattern("d-MMM-yy")
    val lines = scala.io.Source.fromFile(file).getLines().toSeq
    lines.tail.map { line =>
      val cols = line.split(',')
      val date = LocalDate.parse(cols(0), formatter)
      val value = cols(4).toDouble
      (date, value)
    }.reverse.toArray
  }

readGoogleHistory: (file: java.io.File)Array[(java.time.LocalDate, Double)]


In [8]:
files(0)

/home/kranthidr/data/risk_analysis/factors/NYSEARCA%3AGLD.csv

In [9]:
readGoogleHistory(files(0))

[(2004-11-18,44.38), (2004-11-19,44.78), (2004-11-22,44.95), (2004-11-23,44.75), (2004-11-24,45.05), (2004-11-26,45.29), (2004-11-29,45.4), (2004-11-30,45.12), (2004-12-01,45.38), (2004-12-02,44.95), (2004-12-03,45.6), (2004-12-06,45.17), (2004-12-07,45.11), (2004-12-08,44.01), (2004-12-09,43.78), (2004-12-10,43.44), (2004-12-13,43.91), (2004-12-14,43.55), (2004-12-15,43.99), (2004-12-16,43.68), (2004-12-17,44.19), (2004-12-20,44.34), (2004-12-21,44.22), (2004-12-22,44.02), (2004-12-23,44.27), (2004-12-27,44.48), (2004-12-28,44.37), (2004-12-29,43.66), (2004-12-30,43.83), (2004-12-31,43.8), (2005-01-03,43.02), (2005-01-04,42.74), (2005-01-05,42.67), (2005-01-06,42.15), (2005-01-07,41.84), (2005-01-10,41.95), (2005-01-11,42.21), (2005-01-12,42.6), (2005-01-13,42.6), (2005-01-14,42.32), (2005-01-18,42.32), (2005-01-19,42.26), (2005-01-20,42.25), (2005-01-21,42.74), (2005-01-24,42.77), (2005-01-25,42.24), (2005-01-26,42.69), (2005-01-27,42.62), (2005-01-28,42.69), (2005-01-31,42.22), (200

In [10]:
 val allStocks = files.iterator.flatMap { file =>
      try {
        Some(readGoogleHistory(file))
      } catch {
        case e: Exception => None
      }
    }
val rawStocks = allStocks.filter(_.size >= 260 * 5 + 10)

allStocks = non-empty iterator
rawStocks = non-empty iterator


non-empty iterator

In [11]:
rawStocks

non-empty iterator

In [12]:
allStocks

non-empty iterator

In [13]:
val factorsPrefix = "/home/kranthidr/data/risk_analysis/factors/"
val rawFactors = Array("NYSEARCA%3AGLD.csv", "NASDAQ%3ATLT.csv", "NYSEARCA%3ACRED.csv").
      map(x => new File(factorsPrefix + x)).
      map(readGoogleHistory)

factorsPrefix = /home/kranthidr/data/risk_analysis/factors/
rawFactors = Array(Array((2004-11-18,44.38), (2004-11-19,44.78), (2004-11-22,44.95), (2004-11-23,44.75), (2004-11-24,45.05), (2004-11-26,45.29), (2004-11-29,45.4), (2004-11-30,45.12), (2004-12-01,45.38), (2004-12-02,44.95), (2004-12-03,45.6), (2004-12-06,45.17), (2004-12-07,45.11), (2004-12-08,44.01), (2004-12-09,43.78), (2004-12-10,43.44), (2004-12-13,43.91), (2004-12-14,43.55), (2004-12-15,43.99), (2004-12-16,43.68), (2004-12-17,44.19), (2004-12-20,44.34), (2004-12-21,44.22), (2004-12-22,44.02), (2004-12-23,44.27), (2004-12-27,44.48), (2004-12-28,44.37), (2004-12-29,43.66), (2004-12-30,43.83), (2004-12-31,43.8), (2005-01-03,43.02), (2005-01-04,42.74), (2005-01-05,42.67), (2...


[[(2004-11-18,44.38), (2004-11-19,44.78), (2004-11-22,44.95), (2004-11-23,44.75), (2004-11-24,45.05), (2004-11-26,45.29), (2004-11-29,45.4), (2004-11-30,45.12), (2004-12-01,45.38), (2004-12-02,44.95), (2004-12-03,45.6), (2004-12-06,45.17), (2004-12-07,45.11), (2004-12-08,44.01), (2004-12-09,43.78), (2004-12-10,43.44), (2004-12-13,43.91), (2004-12-14,43.55), (2004-12-15,43.99), (2004-12-16,43.68), (2004-12-17,44.19), (2004-12-20,44.34), (2004-12-21,44.22), (2004-12-22,44.02), (2004-12-23,44.27), (2004-12-27,44.48), (2004-12-28,44.37), (2004-12-29,43.66), (2004-12-30,43.83), (2004-12-31,43.8), (2005-01-03,43.02), (2005-01-04,42.74), (2005-01-05,42.67), (2005-01-06,42.15), (2005-01-07,41.84), (2005-01-10,41.95), (2005-01-11,42.21), (2005-01-12,42.6), (2005-01-13,42.6), (2005-01-14,42.32), (2005-01-18,42.32), (2005-01-19,42.26), (2005-01-20,42.25), (2005-01-21,42.74), (2005-01-24,42.77), (2005-01-25,42.24), (2005-01-26,42.69), (2005-01-27,42.62), (2005-01-28,42.69), (2005-01-31,42.22), (20

In [14]:
def trimToRegion(history: Array[(LocalDate, Double)], start: LocalDate, end: LocalDate)
    : Array[(LocalDate, Double)] = {
    var trimmed = history.dropWhile(_._1.isBefore(start)).
      takeWhile(x => x._1.isBefore(end) || x._1.isEqual(end))
    if (trimmed.head._1 != start) {
      trimmed = Array((start, trimmed.head._2)) ++ trimmed
    }
    if (trimmed.last._1 != end) {
      trimmed = trimmed ++ Array((end, trimmed.last._2))
    }
    trimmed
  }

trimToRegion: (history: Array[(java.time.LocalDate, Double)], start: java.time.LocalDate, end: java.time.LocalDate)Array[(java.time.LocalDate, Double)]


In [15]:
  def fillInHistory(history: Array[(LocalDate, Double)], start: LocalDate, end: LocalDate)
    : Array[(LocalDate, Double)] = {
    var cur = history
    val filled = new ArrayBuffer[(LocalDate, Double)]()
    var curDate = start
    while (curDate.isBefore(end)) {
      if (cur.tail.nonEmpty && cur.tail.head._1 == curDate) {
        cur = cur.tail
      }

      filled += ((curDate, cur.head._2))

      curDate = curDate.plusDays(1)
      // Skip weekends
      if (curDate.getDayOfWeek.getValue > 5) {
        curDate = curDate.plusDays(2)
      }
    }
    filled.toArray
  }

fillInHistory: (history: Array[(java.time.LocalDate, Double)], start: java.time.LocalDate, end: java.time.LocalDate)Array[(java.time.LocalDate, Double)]


In [16]:
  def twoWeekReturns(history: Array[(LocalDate, Double)]): Array[Double] = {
    history.sliding(10).map { window =>
      val next = window.last._2
      val prev = window.head._2
      (next - prev) / prev
    }.toArray
  }

  

twoWeekReturns: (history: Array[(java.time.LocalDate, Double)])Array[Double]


In [17]:
 val stocks = rawStocks.map(trimToRegion(_, start, end)).map(fillInHistory(_, start, end))

    val factors = rawFactors.
      map(trimToRegion(_, start, end)).
      map(fillInHistory(_, start, end))

    val stocksReturns = stocks.map(twoWeekReturns).toArray.toSeq
    val factorsReturns = factors.map(twoWeekReturns)

stocks = empty iterator
factors = Array(Array((2009-10-23,103.49), (2009-10-26,101.86), (2009-10-27,101.85), (2009-10-28,100.73), (2009-10-29,102.69), (2009-10-30,102.53), (2009-11-02,103.95), (2009-11-03,106.46), (2009-11-04,107.1), (2009-11-05,106.98), (2009-11-06,107.43), (2009-11-09,108.19), (2009-11-10,108.39), (2009-11-11,109.6), (2009-11-12,108.21), (2009-11-13,109.74), (2009-11-16,111.63), (2009-11-17,111.97), (2009-11-18,112.25), (2009-11-19,112.3), (2009-11-20,112.94), (2009-11-23,114.29), (2009-11-24,114.73), (2009-11-25,116.62), (2009-11-26,116.62), (2009-11-27,115.06), (2009-11-30,115.64), (2009-12-01,117.38), (2009-12-02,119.18), (2009-12-03,118.7), (2009-12-04,113.75), (2009-12-07...


[[(2009-10-23,103.49), (2009-10-26,101.86), (2009-10-27,101.85), (2009-10-28,100.73), (2009-10-29,102.69), (2009-10-30,102.53), (2009-11-02,103.95), (2009-11-03,106.46), (2009-11-04,107.1), (2009-11-05,106.98), (2009-11-06,107.43), (2009-11-09,108.19), (2009-11-10,108.39), (2009-11-11,109.6), (2009-11-12,108.21), (2009-11-13,109.74), (2009-11-16,111.63), (2009-11-17,111.97), (2009-11-18,112.25), (2009-11-19,112.3), (2009-11-20,112.94), (2009-11-23,114.29), (2009-11-24,114.73), (2009-11-25,116.62), (2009-11-26,116.62), (2009-11-27,115.06), (2009-11-30,115.64), (2009-12-01,117.38), (2009-12-02,119.18), (2009-12-03,118.7), (2009-12-04,113.75), (2009-12-07,113.11), (2009-12-08,110.95), (2009-12-09,110.84), (2009-12-10,110.82), (2009-12-11,109.32), (2009-12-14,110.24), (2009-12-15,110.21), (2009-12-16,111.59), (2009-12-17,107.34), (2009-12-18,108.95), (2009-12-21,106.95), (2009-12-22,106.17), (2009-12-23,106.55), (2009-12-24,108.36), (2009-12-25,108.36), (2009-12-28,108.55), (2009-12-29,107

In [18]:
def plotDistribution(samples: Array[Double]): Figure = {
    val min = samples.min
    val max = samples.max
    val stddev = new StatCounter(samples).stdev
    val bandwidth = 1.06 * stddev * math.pow(samples.size, -.2)

    // Using toList before toArray avoids a Scala bug
    val domain = Range.Double(min, max, (max - min) / 100).toList.toArray
    val kd = new KernelDensity().
      setSample(samples.toSeq.toDS.rdd).
      setBandwidth(bandwidth)
    val densities = kd.estimate(domain)
    val f = Figure()
    val p = f.subplot(0)
    p += plot(domain, densities)
    p.xlabel = "Two Week Return ($)"
    p.ylabel = "Density"
    f
  }

plotDistribution: (samples: Array[Double])breeze.plot.Figure


In [19]:
//   def plotDistribution(samples: Dataset[Double]): Figure = {
//     val (min, max, count, stddev) = samples.agg(
//       functions.min($"value"),
//       functions.max($"value"),
//       functions.count($"value"),
//       functions.stddev_pop($"value")
//     ).as[(Double, Double, Long, Double)].first()
//     val bandwidth = 1.06 * stddev * math.pow(count, -.2)

//     // Using toList before toArray avoids a Scala bug
//     val domain = Range.Double(min, max, (max - min) / 100).toList.toArray
//     val kd = new KernelDensity().
//       setSample(samples.rdd).
//       setBandwidth(bandwidth)
//     val densities = kd.estimate(domain)
//     val f = Figure()
//     val p = f.subplot(0)
//     p += plot(domain, densities)
//     p.xlabel = "Two Week Return ($)"
//     p.ylabel = "Density"
//     f
//   }

Name: Syntax Error.
Message: 
StackTrace: 

In [20]:
import scala.collection.JavaConversions._

In [21]:
System.getenv("DISPLAY")

res37: String = null


In [22]:
plotDistribution(factorsReturns(2))

breeze.plot.Figure@208d00cc

In [23]:
// plotDistribution(factorsReturns(3)) 

// Name: java.lang.ArrayIndexOutOfBoundsException
// Message: 3
// StackTrace:

Name: Syntax Error.
Message: 
StackTrace: 

In [24]:
//     val numTrials = 10000000
//     val parallelism = 1000
//     val baseSeed = 1001L

numTrials = 10000000
parallelism = 1000
baseSeed = 1001


1001

In [63]:
    val numTrials = 100
    val parallelism = 10
    val baseSeed = 11L

numTrials = 100
parallelism = 10
baseSeed = 11


11

In [26]:
 def factorMatrix(histories: Seq[Array[Double]]): Array[Array[Double]] = {
    val mat = new Array[Array[Double]](histories.head.length)
    for (i <- histories.head.indices) {
      mat(i) = histories.map(_(i)).toArray
    }
    mat
  }

factorMatrix: (histories: Seq[Array[Double]])Array[Array[Double]]


In [27]:
  def featurize(factorReturns: Array[Double]): Array[Double] = {
    val squaredReturns = factorReturns.map(x => math.signum(x) * x * x)
    val squareRootedReturns = factorReturns.map(x => math.signum(x) * math.sqrt(math.abs(x)))
    squaredReturns ++ squareRootedReturns ++ factorReturns
  }

featurize: (factorReturns: Array[Double])Array[Double]


In [28]:

  /**
   * Calculate the return of a particular instrument under particular trial conditions.
   */
  def instrumentTrialReturn(instrument: Array[Double], trial: Array[Double]): Double = {
    var instrumentTrialReturn = instrument(0)
    var i = 0
    while (i < trial.length) {
      instrumentTrialReturn += trial(i) * instrument(i+1)
      i += 1
    }
    instrumentTrialReturn
  }

instrumentTrialReturn: (instrument: Array[Double], trial: Array[Double])Double


In [31]:
 

  def linearModel(instrument: Array[Double], factorMatrix: Array[Array[Double]])
    : OLSMultipleLinearRegression = {
    val regression = new OLSMultipleLinearRegression()
    regression.newSampleData(instrument, factorMatrix)
    regression
  }


linearModel: (instrument: Array[Double], factorMatrix: Array[Array[Double]])org.apache.commons.math3.stat.regression.OLSMultipleLinearRegression


In [32]:
 def computeFactorWeights(
    stocksReturns: Seq[Array[Double]],
    factorFeatures: Array[Array[Double]]): Array[Array[Double]] = {
    stocksReturns.map(linearModel(_, factorFeatures)).map(_.estimateRegressionParameters()).toArray
  }

computeFactorWeights: (stocksReturns: Seq[Array[Double]], factorFeatures: Array[Array[Double]])Array[Array[Double]]


In [33]:
  
  def fivePercentVaR(trials: Dataset[Double]): Double = {
    val quantiles = trials.stat.approxQuantile("value", Array(0.05), 0.0)
    quantiles.head
  }

fivePercentVaR: (trials: org.apache.spark.sql.Dataset[Double])Double


In [34]:
  def fivePercentCVaR(trials: Dataset[Double]): Double = {
    val topLosses = trials.orderBy("value").limit(math.max(trials.count().toInt / 20, 1))
    topLosses.agg("value" -> "avg").first()(0).asInstanceOf[Double]
  }

fivePercentCVaR: (trials: org.apache.spark.sql.Dataset[Double])Double


In [35]:
  def countFailures(stocksReturns: Seq[Array[Double]], valueAtRisk: Double): Int = {
    var failures = 0
    for (i <- stocksReturns.head.indices) {
      val loss = stocksReturns.map(_(i)).sum
      if (loss < valueAtRisk) {
        failures += 1
      }
    }
    failures
  }

countFailures: (stocksReturns: Seq[Array[Double]], valueAtRisk: Double)Int


In [36]:
  def kupiecTestStatistic(total: Int, failures: Int, confidenceLevel: Double): Double = {
    val failureRatio = failures.toDouble / total
    val logNumer = (total - failures) * math.log1p(-confidenceLevel) +
      failures * math.log(confidenceLevel)
    val logDenom = (total - failures) * math.log1p(-failureRatio) +
      failures * math.log(failureRatio)
    -2 * (logNumer - logDenom)
  }

kupiecTestStatistic: (total: Int, failures: Int, confidenceLevel: Double)Double


In [37]:
  def kupiecTestPValue(
      stocksReturns: Seq[Array[Double]],
      valueAtRisk: Double,
      confidenceLevel: Double): Double = {
    val failures = countFailures(stocksReturns, valueAtRisk)
    val total = stocksReturns.head.length
    val testStatistic = kupiecTestStatistic(total, failures, confidenceLevel)
    1 - new ChiSquaredDistribution(1.0).cumulativeProbability(testStatistic)
  }

kupiecTestPValue: (stocksReturns: Seq[Array[Double]], valueAtRisk: Double, confidenceLevel: Double)Double


In [38]:
//   def bootstrappedConfidenceInterval(
//       trials: Dataset[Double],
//       computeStatistic: Dataset[Double] => Double,
//       numResamples: Int,
//       probability: Double): (Double, Double) = {
      
//     val stats = (0 until numResamples).map { i =>
//       val resample = trials.sample(true, 1.0)
//       computeStatistic(resample)
//     }.sorted
      
//     val lowerIndex = (numResamples * probability / 2 - 1).toInt
      
//     val upperIndex = math.ceil(numResamples * (1 - probability / 2)).toInt
//     (stats(lowerIndex), stats(upperIndex))
//   }

Name: Syntax Error.
Message: 
StackTrace: 

In [29]:

  /**
   * Calculate the full return of the portfolio under particular trial conditions.
   */
  def trialReturn(trial: Array[Double], instruments: Seq[Array[Double]]): Double = {
    var totalReturn = 0.0
    for (instrument <- instruments) {
      totalReturn += instrumentTrialReturn(instrument, trial)
    }
    totalReturn / instruments.size
  }

trialReturn: (trial: Array[Double], instruments: Seq[Array[Double]])Double


In [74]:
  def trialReturns(
      seed: Long,
      numTrials: Int,
      instruments: Seq[Array[Double]],
      factorMeans: Array[Double],
      factorCovariances: Array[Array[Double]]): Array[Double] = { 
      //Seq[Double]
      
    val rand = new MersenneTwister(seed)
      
    val multivariateNormal = new MultivariateNormalDistribution(rand, factorMeans,
      factorCovariances)

    val trialReturns = new Array[Double](numTrials)
      
    for (i <- 0 until numTrials) {
      val trialFactorReturns = multivariateNormal.sample()
      val trialFeatures = featurize(trialFactorReturns)
      trialReturns(i) = trialReturn(trialFeatures, instruments)
    }
      
    trialReturns.toArray
  }

lastException: Throwable = null
trialReturns: (seed: Long, numTrials: Int, instruments: Seq[Array[Double]], factorMeans: Array[Double], factorCovariances: Array[Array[Double]])Array[Double]


In [75]:
import org.apache.spark.sql.UDFRegistration

In [76]:
 def computeTrialReturns(
      stocksReturns: Seq[Array[Double]],
      factorsReturns: Seq[Array[Double]],
      baseSeed: Long,
      numTrials: Int,
      parallelism: Int): Dataset[Double] = {
    val factorMat = factorMatrix(factorsReturns)
    val factorCov = new Covariance(factorMat).getCovarianceMatrix().getData()
    val factorMeans = factorsReturns.map(factor => factor.sum / factor.size).toArray
    val factorFeatures = factorMat.map(featurize)
    val factorWeights = computeFactorWeights(stocksReturns, factorFeatures)

    // Generate different seeds so that our simulations don't all end up with the same results
    val seeds = (baseSeed until baseSeed + parallelism)
    val seedDS = seeds.toDS().repartition(parallelism)

    // Main computation: run simulations and compute aggregate return for each
    seedDS.flatMap(trialReturns(_, numTrials / parallelism, factorWeights, factorMeans, factorCov))
  }

computeTrialReturns: (stocksReturns: Seq[Array[Double]], factorsReturns: Seq[Array[Double]], baseSeed: Long, numTrials: Int, parallelism: Int)org.apache.spark.sql.Dataset[Double]


In [77]:
 val factorMat = factorMatrix(factorsReturns)
    val factorCov = new Covariance(factorMat).getCovarianceMatrix().getData()
    val factorMeans = factorsReturns.map(factor => factor.sum / factor.size).toArray
    val factorFeatures = factorMat.map(featurize)
    val factorWeights = computeFactorWeights(stocksReturns, factorFeatures)

factorMat = Array(Array(0.033723065030437815, -0.018009478672985864, 0.006638264143465644), Array(0.05468289809542517, -0.004161331626120365, 0.009822403016172187), Array(0.06224840451644579, -0.01642105263157897, 0.001774098166765295), Array(0.07604487243125183, -0.02283918281822951, -0.0018691588785047903), Array(0.06728990164572983, -0.00794407372100413, 0.0012790240062966888), Array(0.0553984199746415, -0.023908958028816103, -0.002553776642765987), Array(0.055699855699855624, -0.011888479747501268, 0.0027510316368638352), Array(0.0485628405034755, 0.009776833156216809, 0.008289746373235995), Array(0.045471521942110224, 0.02456027456027463, 0.007853917141174133), Array(0.04926154421387171, 0.0185542685542686, 0.006594488188976395), Array(0.045331843991436194, 0....


[[0.033723065030437815, -0.018009478672985864, 0.006638264143465644], [0.05468289809542517, -0.004161331626120365, 0.009822403016172187], [0.06224840451644579, -0.01642105263157897, 0.001774098166765295], [0.07604487243125183, -0.02283918281822951, -0.0018691588785047903], [0.06728990164572983, -0.00794407372100413, 0.0012790240062966888], [0.0553984199746415, -0.023908958028816103, -0.002553776642765987], [0.055699855699855624, -0.011888479747501268, 0.0027510316368638352], [0.0485628405034755, 0.009776833156216809, 0.008289746373235995], [0.045471521942110224, 0.02456027456027463, 0.007853917141174133], [0.04926154421387171, 0.0185542685542686, 0.006594488188976395], [0.045331843991436194, 0.019393549769634654, 0.001965022597759902], [0.04390424253627877, 0.017979452054794592, 0.0017709563164107892], [0.054433065780976155, 0.018548300632572148, 0.007786319731914117], [0.04680656934306578, 0.020713218022635038, 0.008352166650289953], [0.07771924960724527, 0.02727564445395242, 0.012900

In [78]:
val seeds = (baseSeed until baseSeed + parallelism)
    val seedDS = seeds.toDS().repartition(parallelism)

seeds = NumericRange(11, 12, 13, 14, 15, 16, 17, 18, 19, 20)
seedDS = [value: bigint]


[value: bigint]

In [79]:
seedDS.limit(3).show()

+-----+
|value|
+-----+
|   20|
|   11|
|   17|
+-----+



In [80]:
trialReturns(seedDS.first, numTrials / parallelism, factorWeights, factorMeans, factorCov)

[0.008568446160079094, 4.1313170269107374E-4, 0.0030641902472235295, 0.020240446442405375, -0.006205811759520545, 0.015093046141827242, -0.0011216019001951995, 0.0245922269357368, 0.004147711194380114, -0.009992380002792215]

In [81]:
trialReturns(seedDS.first, numTrials / parallelism, factorWeights, factorMeans, factorCov).toList.toArray

[0.008568446160079094, 4.1313170269107374E-4, 0.0030641902472235295, 0.020240446442405375, -0.006205811759520545, 0.015093046141827242, -0.0011216019001951995, 0.0245922269357368, 0.004147711194380114, -0.009992380002792215]

In [86]:
  def multiply(
      x: Long): Long = {
   x*x
  }
seedDS.limit(3).map(x => multiply(x)).take(3)

multiply: (x: Long)Long


[400, 121, 289]

In [87]:
 seedDS.limit(3).map(x => trialReturns(x, numTrials / parallelism, factorWeights, factorMeans, factorCov)).take(1)

Name: org.apache.spark.SparkException
Message: Task not serializable
StackTrace:   at org.apache.spark.util.ClosureCleaner$.ensureSerializable(ClosureCleaner.scala:345)
  at org.apache.spark.util.ClosureCleaner$.org$apache$spark$util$ClosureCleaner$$clean(ClosureCleaner.scala:335)
  at org.apache.spark.util.ClosureCleaner$.clean(ClosureCleaner.scala:159)
  at org.apache.spark.SparkContext.clean(SparkContext.scala:2299)
  at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsWithIndex$1.apply(RDD.scala:844)
  at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsWithIndex$1.apply(RDD.scala:843)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
  at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
  at org.apache.spark.rdd.RDD.mapPartitionsWithIndex(RDD.scala:843)
  at org.apache.spark.sql.execution.WholeStageCodegenExec.doExecute(WholeStageCodegenExec.scala:608)
  at org.apache

In [39]:
    val trials = computeTrialReturns(stocksReturns, factorsReturns, baseSeed, numTrials,parallelism)
    

trials = [value: double]


[value: double]

In [45]:
  
    trials.cache()
   

[value: double]

In [46]:
trials.first

Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task not serializable: java.io.NotSerializableException: scala.collection.Iterator$$anon$12
Serialization stack:
	- object not serializable (class: scala.collection.Iterator$$anon$12, value: empty iterator)
	- field (class: $line38.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw, name: allStocks, type: interface scala.collection.Iterator)
	- object (class $line38.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw, $line38.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw@4d0d766)
	- field (class: $line38.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw, name: $iw, type: class $line38.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw)
	- object (class $line38.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw, $line38.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw@13ea21c8)
	- field (class: $line38.$read$$iw$$iw$$iw$$iw$$iw$$iw, name: $iw, type: class $line38.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw)
	- object (class $line38.$read$$iw$$iw$$iw$$iw$$iw$$iw, $line38.$read$$iw$$iw$$iw$$iw$$iw$$iw@5a68cd4f)
	

In [ ]:
   
    val valueAtRisk = fivePercentVaR(trials)
    val conditionalValueAtRisk = fivePercentCVaR(trials)
 

In [ ]:
     
    println("VaR 5%: " + valueAtRisk)
    println("CVaR 5%: " + conditionalValueAtRisk)
      
    val varConfidenceInterval = bootstrappedConfidenceInterval(trials,fivePercentVaR, 100, .05)

       val stats = (0 until numResamples).map { i =>
      val resample = trials.sample(true, 1.0)
      computeStatistic(resample)
    }.sorted
      
    val lowerIndex = (numResamples * probability / 2 - 1).toInt
      
    val upperIndex = math.ceil(numResamples * (1 - probability / 2)).toInt
      
    (stats(lowerIndex), stats(upperIndex))

In [ ]:
    val cvarConfidenceInterval = bootstrappedConfidenceInterval(trials, fivePercentCVaR, 100, .05)
      

In [ ]:
    println("VaR confidence interval: " + varConfidenceInterval)

In [ ]:
    println("CVaR confidence interval: " + cvarConfidenceInterval)
   

In [ ]:
   
    println("Kupiec test p-value: " + kupiecTestPValue(stocksReturns, valueAtRisk, 0.05))
      
 //   runRisk.plotDistribution(trials)